In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [3]:
stats_equipos = pd.read_csv("./partidos_join11_04_20_clean.csv")

In [4]:
stats_equipos

,boxcore_url,Date,Season,local_team,visitor_team,local_pts,visitor_pts,Result,local_Lose,local_Win,...,visitor_orb,visitor_orb_pct,visitor_pf,visitor_stl,visitor_stl_pct,visitor_tov,visitor_tov_pct,visitor_trb_pct,visitor_trn,visitor_true_shooting_pct
0,/boxscores/200910300PHI.html,2009Oct30,2010.0,Philadelphia 76ers,Milwaukee Bucks,99,86,W,1.0,0.0,...,12,23.5,27,7,7.1,20,17.6,49.4,44,0.461
1,/boxscores/200911030PHI.html,2009Nov3,2010.0,Philadelphia 76ers,Boston Celtics,74,105,L,1.0,2.0,...,7,23.3,21,10,11.3,17,17.9,53.4,39,0.672
2,/boxscores/200911060PHI.html,2009Nov6,2010.0,Philadelphia 76ers,New Jersey Nets,97,94,W,2.0,2.0,...,12,27.9,24,6,6.8,13,12.5,50.0,41,0.514
3,/boxscores/200911090PHI.html,2009Nov9,2010.0,Philadelphia 76ers,Phoenix Suns,115,119,L,3.0,3.0,...,6,17.1,18,4,4.3,12,11.8,45.8,33,0.663
4,/boxscores/200911130PHI.html,2009Nov13,2010.0,Philadelphia 76ers,Utah Jazz,90,112,L,4.0,4.0,...,13,35.1,15,9,10.2,12,11.5,57.1,44,0.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229,/boxscores/202002220CHO.html,2020Feb22,2020.0,Charlotte Hornets,Brooklyn Nets,86,115,L,36.0,19.0,...,9,20.0,20,10,10.4,10,9.5,54.5,54,0.603
12230,/boxscores/202002260CHO.html,2020Feb26,2020.0,Charlotte Hornets,New York Knicks,107,101,W,38.0,19.0,...,13,32.5,24,5,5.3,15,14.0,53.2,42,0.546
12231,/boxscores/202003030CHO.html,2020Mar3,2020.0,Charlotte Hornets,San Antonio Spurs,103,104,L,39.0,21.0,...,6,15.8,16,12,13.2,10,10.1,41.8,33,0.586
12232,/boxscores/202003050CHO.html,2020Mar5,2020.0,Charlotte Hornets,Denver Nuggets,112,114,L,40.0,21.0,...,13,34.2,21,7,7.8,12,11.6,52.8,38,0.621


In [5]:
stats_equipos['number_date'] = stats_equipos['boxcore_url'].replace('[A-z]', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\.', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\/', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].str.slice(0,8)

In [6]:
stats_equipos['number_date'] = pd.to_datetime(stats_equipos['number_date'], format='%Y%m%d', errors='ignore')

In [7]:
stats_equipos['previous_date'] = stats_equipos['number_date'] - timedelta(days =1)
stats_equipos['previous_date'] = pd.to_datetime(stats_equipos['previous_date'], format='%Y%m%d', errors='ignore')

In [8]:
stats_equipos['local_played_previous_date'] = 0
stats_equipos['visitor_played_previous_date'] = 0
stats_equipos['previous_10_days'] = stats_equipos['number_date'] - timedelta(days =10)
stats_equipos['previous_10_days'] = pd.to_datetime(stats_equipos['previous_10_days'], format='%Y%m%d', errors='ignore')
stats_equipos['local_played_local'] = 0
stats_equipos['local_played_visitor'] = 0
stats_equipos['visitor_played_visitor'] = 0
stats_equipos['visitor_played_local'] = 0

for actual in stats_equipos.index:
    
    jugo_dia_anterior = stats_equipos[(stats_equipos ['number_date'] == stats_equipos.loc[actual,'previous_date'])]
    
    visitor_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'visitor_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]
    local_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'local_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]
    
    diez_dias_previos = stats_equipos[(stats_equipos['number_date'] > stats_equipos.loc[actual,'previous_10_days']) & (stats_equipos['number_date'] < (stats_equipos.loc[actual,'number_date']))]
    
    if not(local_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'local_played_previous_date'] = 1
   
    if not (visitor_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'visitor_played_previous_date'] = 1
    
    if not (diez_dias_previos.empty):                                 

        local_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'local_team']))]        
        local_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]   
        visitante_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'visitor_team']))]
        visitante_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]

        if not (local_jugo_local.empty | local_jugo_visitante.empty):
            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]    
            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')
            
            if (maximo_local_jugo_local.index.max() > maximo_local_jugo_visitante.index.max()):

                if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                    stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_local'] = 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0

            elif (maximo_local_jugo_local.index.max() < maximo_local_jugo_visitante.index.max()):

                if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                    stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_visitor'] = 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                    
        elif not (local_jugo_local.empty):

            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            
            if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
            else:
                stats_equipos.loc[actual,'local_played_local'] = 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
        
        elif not (local_jugo_visitante.empty):

            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')

            if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                stats_equipos.loc[actual,'local_played_local'] = 0
            else:
                stats_equipos.loc[actual,'local_played_visitor'] = 1
                stats_equipos.loc[actual,'local_played_local'] = 0            
        
        if not (visitante_jugo_local.empty | visitante_jugo_visitante.empty):

            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')

            if (maximo_visitante_jugo_local.index.max() > maximo_visitante_jugo_visitante.index.max()):
                
                if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0

            elif (maximo_visitante_jugo_local.index.max() < maximo_visitante_jugo_visitante.index.max()):
                if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                    stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                    
        elif not (visitante_jugo_local.empty):
            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            
            if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
            else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                    
        elif not (visitante_jugo_visitante.empty):
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')
            
            if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0
            else:
                stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0

C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [9]:
stats_equipos[(stats_equipos['Season'] == 2010.0) & ((stats_equipos['local_team'] == 'Philadelphia 76ers') | (stats_equipos['visitor_team'] == 'Philadelphia 76ers'))].sort_values('number_date').head(20)

,boxcore_url,Date,Season,local_team,visitor_team,local_pts,visitor_pts,Result,local_Lose,local_Win,...,visitor_true_shooting_pct,number_date,previous_date,local_played_previous_date,visitor_played_previous_date,previous_10_days,local_played_local,local_played_visitor,visitor_played_visitor,visitor_played_local
545,/boxscores/200910280ORL.html,2009Oct28,2010.0,Orlando Magic,Philadelphia 76ers,120,106,W,0.0,0.0,...,0.581,2009-10-28,2009-10-27,0,0,2009-10-18,0,0,0,0
0,/boxscores/200910300PHI.html,2009Oct30,2010.0,Philadelphia 76ers,Milwaukee Bucks,99,86,W,1.0,0.0,...,0.461,2009-10-30,2009-10-29,0,0,2009-10-20,0,1,0,0
156,/boxscores/200910310NYK.html,2009Oct31,2010.0,New York Knicks,Philadelphia 76ers,127,141,L,2.0,0.0,...,0.688,2009-10-31,2009-10-30,1,1,2009-10-21,0,1,0,1
1,/boxscores/200911030PHI.html,2009Nov3,2010.0,Philadelphia 76ers,Boston Celtics,74,105,L,1.0,2.0,...,0.672,2009-11-03,2009-11-02,0,0,2009-10-24,0,1,0,1
2,/boxscores/200911060PHI.html,2009Nov6,2010.0,Philadelphia 76ers,New Jersey Nets,97,94,W,2.0,2.0,...,0.514,2009-11-06,2009-11-05,0,0,2009-10-27,1,0,0,1
40,/boxscores/200911080DET.html,2009Nov8,2010.0,Detroit Pistons,Philadelphia 76ers,88,81,W,4.0,2.0,...,0.476,2009-11-08,2009-11-07,0,0,2009-10-29,0,1,0,2
3,/boxscores/200911090PHI.html,2009Nov9,2010.0,Philadelphia 76ers,Phoenix Suns,115,119,L,3.0,3.0,...,0.663,2009-11-09,2009-11-08,1,1,2009-10-30,0,1,1,0
314,/boxscores/200911110NJN.html,2009Nov11,2010.0,New Jersey Nets,Philadelphia 76ers,79,82,L,7.0,0.0,...,0.452,2009-11-11,2009-11-10,0,0,2009-11-01,1,0,0,1
4,/boxscores/200911130PHI.html,2009Nov13,2010.0,Philadelphia 76ers,Utah Jazz,90,112,L,4.0,4.0,...,0.605,2009-11-13,2009-11-12,0,0,2009-11-03,0,1,1,0
82,/boxscores/200911140CHI.html,2009Nov14,2010.0,Chicago Bulls,Philadelphia 76ers,94,88,W,4.0,4.0,...,0.488,2009-11-14,2009-11-13,0,1,2009-11-04,0,1,0,1


In [ ]:
stats_equipos[(stats_equipos['Season'] == 2010.0) & (stats_equipos['visitor_team'] == 'Philadelphia 76ers')].sort_values('number_date').head(10)

In [ ]:
stats_equipos